# Amazon Bedrock モデルを使用した LangGraph エージェントを Amazon Bedrock AgentCore ランタイムでホスティングする

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore ランタイムを使用して既存のエージェントをホストする方法を学びます。

Amazon Bedrock モデルの LangGraph の例に焦点を当てます。Amazon Bedrock モデルを使用した Strands エージェントについては[こちら](../01-strands-with-bedrock-model)を、OpenAI モデルを使用した Strands エージェントについては[こちら](../03-strands-with-openai-model)を参照してください。

### チュートリアルの詳細

| 情報                | 詳細                                                                          |
|:--------------------|:------------------------------------------------------------------------------|
| チュートリアルの種類 | 会話型                                                                        |
| エージェントの種類   | 単一                                                                          |
| エージェントフレームワーク | LangGraph                                                                     |
| LLM モデル          | Anthropic Claude Sonnet 3                                                     |
| チュートリアルコンポーネント | AgentCore ランタイムでのエージェントのホスティング。LangGraph と Amazon Bedrock モデルの使用 |
| チュートリアルの分野 | 分野横断                                                                      |
| 例の複雑さ          | 簡単                                                                          |
| 使用した SDK        | Amazon BedrockAgentCore Python SDK と boto3                                    |

### チュートリアルのアーキテクチャ

このチュートリアルでは、既存のエージェントを AgentCore ランタイムにデプロイする方法を説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した LangGraph エージェントを使用します。

私たちの例では、`get_weather` と `get_time` の 2 つのツールを持つ非常に単純なエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore ランタイムでのエージェントのホスティング
* Amazon Bedrock モデルの使用
* LangGraph の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です:

* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker の実行

In [ ]:
#!uv add -r requirements.txt --active

日本語訳:

このコマンドは、 requirements.txt ファイルに記載された依存関係をインストールするために使用されます。 --active オプションは、現在のアクティブな仮想環境にパッケージをインストールすることを指示しています。

## エージェントの作成とローカルでの実験

AgentCore Runtimeにエージェントをデプロイする前に、実験目的でローカルでエージェントを開発して実行しましょう。

本番のエージェントアプリケーションでは、エージェントの作成プロセスとエージェントの呼び出しプロセスを分離する必要があります。AgentCore Runtimeでは、エージェントの呼び出し部分に `@app.entrypoint` デコレータを付けて、ランタイムのエントリポイントとします。まずは、実験段階でそれぞれのエージェントがどのように開発されるかを見ていきましょう。

ここでのアーキテクチャは次のようになります。

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math

# 計算ツールを作成

日本語訳:

# 計算ツールを作成する

まず、 Python の組み込みモジュールである `math` をインポートします。このモジュールには、三角関数や対数関数、平方根などの数学関数が含まれています。

```python
import math
```

次に、ユーザーに入力を求める関数 `get_input()` を定義します。この関数は、ユーザーに 2 つの数値と演算子を入力するよう求め、入力された値を返します。

```python
def get_input():
    num1 = float(input(" 最初の数値を入力してください: "))
    op = input(" 演算子を入力してください (+, -, *, /): ")
    num2 = float(input(" 2 番目の数値を入力してください: "))
    return num1, op, num2
```

次に、実際の計算を行う `calculate()` 関数を定義します。この関数は、入力された 2 つの数値と演算子に基づいて計算を行い、結果を返します。

```python
def calculate(num1, op, num2):
    if op == "+":
        return num1 + num2
    elif op == "-":
        return num1 - num2
    elif op == "*":
        return num1 * num2
    elif op == "/":
        if num2 != 0:
            return num1 / num2
        else:
            return "ゼロ除算エラー"
    else:
        return "無効な演算子"
```

最後に、メイン関数を定義し、 `get_input()` と `calculate()` 関数を呼び出します。計算結果を出力します。

```python
def main():
    num1, op, num2 = get_input()
    result = calculate(num1, op, num2)
    print(f" 計算結果: {num1} {op} {num2} = {result}")

if __name__ == "__main__":
    main()
```

このプログラムを実行すると、ユーザーに数値と演算子の入力を求められます。入力された値に基づいて計算が行われ、結果が出力されます。
@tool
def calculator(expression: str) -> str:
    ```
数学的な式の結果を計算します。

Args:
    expression: 文字列として与えられた数学的な式 (例: "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")

Returns:
    計算結果を文字列で返します
```
    try:
        # 式で使用できる安全な関数を定義

 var safeFunction = function ( a, b ) {
     return a + b ;
 } ;

 var unsafeFunction = function ( a, b ) {
     return a * b ; // この関数は安全ではありません
 }

 // 安全な関数のみを許可するホワイトリスト
 var whitelist = [ 'safeFunction' ] ;

 // 式を評価する前に、使用される関数がホワイトリストに含まれているかチェックします
 function isSafe ( path ) {
     return whitelist.indexOf ( path.split ( '.' ) [ 0 ] ) !== -1 ;
 }
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # 数学関数

日本語訳:

この行は、コメントとして機能し、次のセクションのタイトルを示しています。

以下の関数は、さまざまな数学的な計算を実行するために使用されます。

def add(x, y):
    """
    2 つの数 x と y を加算し、その結果を返します。
    """
    return x + y

def subtract(x, y):
    """
    数 y を数 x から引き、その結果を返します。
    """
    return x - y

def multiply(x, y):
    """
    2 つの数 x と y を乗算し、その結果を返します。
    """
    return x * y

def divide(x, y):
    """
    数 x を数 y で割り、その結果を返します。
    ゼロ除算の場合は ValueError を発生させます。
    """
    if y == 0:
        raise ValueError(' 0 で割ることはできません。')
    return x / y

# 使用例
print(add(2, 3))  # 5
print(subtract(5, 3))  # 2
print(multiply(6, 7))  # 42
print(divide(15, 3))  # 5.0
print(divide(1, 0))  # ValueError: 0 で割ることはできません。

注: コードやコマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しています。半角英数字の前後には半角スペースを挿入しています。
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # 基本演算子 (明示的な使用のため)

 a = 5
 b = 10

 # 算術演算子
 print('a + b =', a + b)  # 15
 print('a - b =', a - b)  # -5
 print('a * b =', a * b)  # 50
 print('a / b =', a / b)  # 0.5
 print('a // b =', a // b)  # 0 (床除算)
 print('a % b =', a % b)  # 5 (剰余)
 print('a ** b =', a ** b)  # 9765625 (累乗)

 # 比較演算子
 print('a > b =', a > b)  # False
 print('a < b =', a < b)  # True
 print('a == b =', a == b)  # False
 print('a != b =', a != b)  # True
 print('a >= b =', a >= b)  # False
 print('a <= b =', a <= b)  # True

 # 代入演算子
 a = 10
 b = 20
 a += b  # a = a + b
 print('a =', a)  # 30
 a -= b  # a = a - b
 print('a =', a)  # 10
 a *= b  # a = a * b
 print('a =', a)  # 200

 # ビット演算子
 a = 0b10101010
 b = 0b01010101
 print('a =', bin(a), 'b =', bin(b))
 print('a & b =', bin(a & b))  # 0b00000000
 print('a | b =', bin(a | b))  # 0b11111111
 print('a ^ b =', bin(a ^ b))  # 0b11111111
 print('~a =', bin(~a))  # -0b10101011
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # 式を安全に評価する

日本語訳:

この関数は、与えられた文字列を安全に評価します。まず、 `ast.parse()` を使って文字列を抽象構文木 (AST) に変換します。次に、 `ast.walk()` を使って AST を走査し、危険な構文要素 (外部モジュールのインポートなど) がないかチェックします。危険な要素が見つからない場合は、 `eval()` を使って式を評価し、その結果を返します。それ以外の場合は、 `ValueError` 例外をスローします。

この関数は、信頼できない入力を安全に評価する必要がある場合に使用できます。たとえば、ユーザーが入力した数式を評価したり、設定ファイルから読み込んだ式を評価したりする場合などです。

使用例:

```python
result = safe_eval('2 + 3 * 4')  # 結果は 14
malicious = safe_eval('import os; os.system("rm -rf /")')  # ValueError が発生
```
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# 独自の天気ツールを作成する

日本語訳:

独自の天気ツールを作成するには、以下の手順に従ってください。

1. まず、気象データを取得するための API キーを取得する必要があります。 OpenWeatherMap や Dark Sky などの天気 API プロバイダから API キーを取得できます。

2. 次に、プログラミング言語 (Python、JavaScript など) を選択し、API から天気データを取得するコードを記述します。たとえば、Python の場合は requests ライブラリを使用して API にリクエストを送信できます。

```python
import requests

api_key = "YOUR_API_KEY"
city = "Tokyo"
url = f"http://api.openweathermap.org/data/2.5/weather?q={ city }&appid={ api_key }"

response = requests.get(url)
data = response.json()
```

3. 取得した天気データを処理し、必要な情報 (気温、湿度、天気概況など) を抽出します。

4. 最後に、抽出した情報を表示するユーザーインターフェイスを作成します。ウェブアプリ、デスクトップアプリ、モバイルアプリなど、目的に合わせて適切なインターフェイスを選択します。

この手順を踏めば、独自の天気ツールを作成できます。天気データの取得方法、データの処理方法、ユーザーインターフェイスのデザインなど、さまざまなカスタマイズが可能です。
@tool
def weather():
    以下が日本語訳になります。

"""Get weather"""

「 天気を取得する 」

技術的な用語は翻訳せず、そのまま残しました。半角英数字の前後には半角スペースを挿入しています。  # Dummy implementation

# ダミー実装

import random

def roll_dice( num_rolls, num_sides ):
    """
    ダイスを num_rolls 回転がして、それぞれの出目を num_sides 面のダイスから取得します。
    出目のリストを返します。
    """
    rolls = [ ]
    for _ in range( num_rolls ):
        rolls.append( random.randint( 1, num_sides ) )
    return rolls

def main( ):
    """
    メイン実行部分
    """
    print( "シミュレーションゲームへようこそ!" )
    print( "コンピュータはダミーの実装を使用しています。" )
    print( )

    # ゲームのセットアップ
    num_players = 2
    player_names = [ ]
    for i in range( num_players ):
        name = input( f"プレイヤー { i + 1 } の名前を入力してください: " )
        player_names.append( name )

    # ゲームのループ
    while True:
        # プレイヤーごとにダイスを振る
        for name in player_names:
            num_rolls = random.randint( 1, 6 )
            num_sides = 6
            rolls = roll_dice( num_rolls, num_sides )
            print( f"{ name } は { num_rolls } 個の { num_sides } 面ダイスを振りました: { rolls }" )

        # ゲームを続けるかどうかを尋ねる
        response = input( "ゲームを続けますか? (y/n): " )
        if response.lower( ) != "y":
            break

    print( "ゲームを終了します。お疲れさまでした!" )

if __name__ == "__main__":
    main( )
    return "sunny"

# エージェントを手動の LangGraph 構築を使って定義する

日本語訳:

エージェントを手動の LangGraph 構築を使って定義します。

import langchain
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain import SerpAPIWrapper

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="searches the internet for the query provided and returns the results",
    )
]

agent = initialize_agent(tools, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, handle_parsing_errors=True)
def create_agent():
    以下が日本語訳になります。

"""LangGraph エージェントの作成と設定"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)

大規模言語モデル (LLM) を初期化します (モデルとパラメーターは必要に応じて調整してください)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model

日本語訳:
# または好みの model を使用してください。
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM

# ツールを LLM にバインドする

To allow the LLM to interact with tools, we first need to define a list of tools and their corresponding functions. Here's an example of how we can define a Python REPL tool:

LLM がツールと対話できるようにするには、まず、ツールのリストとそれに対応する関数を定義する必要があります。Python REPL ツールを定義する例は次のとおりです。

```python
import re
import sys
from typing import Any

def python_repl(input_string: str) -> str:
    """Python REPL that executes the input string and returns the output."""
    try:
        output = eval(input_string, sys.modules)
        if output is not None:
            return str(output)
        else:
            return ""
    except Exception as e:
        return str(e)

tools = [
    Tool(
        name="Python REPL",
        description="A Python REPL that allows executing Python code. Useful for simple calculations, data manipulation, or exploring Python libraries. Input should be a valid Python expression.",
        func=python_repl,
    )
]
```

We define a `python_repl` function that takes an input string, evaluates it as Python code, and returns the output. We then create a `Tool` object with a name, description, and the `python_repl` function.

`python_repl` 関数は入力文字列を受け取り、Python コードとして評価し、出力を返します。次に、名前、説明、および `python_repl` 関数を持つ `Tool` オブジェクトを作成します。

To bind these tools to the LLM, we can use the `llm.add_tools` method:

これらのツールを LLM にバインドするには、`llm.add_tools` メソッドを使用できます。

```python
from langchain.llms import OpenAI
from langchain.agents import initialize_agent

llm = OpenAI(temperature=0)
llm.add_tools(tools)
agent = initialize_agent(llm, tools, llm_prefix="Python REPL", max_iterations=3, early_stopping_method="force")
```

This creates an agent that can use the Python REPL tool. We can then call the agent with a query:

これにより、Python REPL ツールを使用できるエージェントが作成されます。次に、クエリでエージェントを呼び出すことができます。

```python
query = "What is 3 * 4 * 5?"
result = agent.run(query)
print(result)
```

This will output:

```
Python REPL: 3 * 4 * 5
Output: 60
```

The agent recognized that the query could be answered by executing Python code, so it used the Python REPL tool to calculate the result.

エージェントはクエリが Python コードを実行することで答えられると認識したため、Python REPL ツールを使用して結果を計算しました。
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message

システムメッセージ

The following command will install the required dependencies:

以下の コマンド を実行すると、必要な 依存関係 をインストールできます:

```
pip install numpy pandas matplotlib
```

Don't forget to activate your virtual environment (e.g. `source env/bin/activate`) before running the `pip install` command.

`pip install` コマンドを実行する前に、仮想環境を有効化することを忘れずに (例: `source env/bin/activate`)。

After installing the dependencies, you can run the `main.py` script:

依存関係をインストールした後、 `main.py` スクリプトを実行できます:

```
python main.py
```

This will load the dataset, perform some analysis, and generate a few plots. Let me know if you have any other questions!

これにより、データセットが読み込まれ、いくつかの分析が実行され、プロットが生成されます。他に質問があれば、お尋ねください。
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node

チャットボットノードを定義します。

import random

class ChatbotNode:
    """
    A chatbot node that generates a response based on the input message.
    """

    def __init__(self, messages, responses):
        """
        Initialize the chatbot node with a list of messages and responses.
        """
        self.messages = messages
        self.responses = responses

    def get_response(self, message):
        """
        Get a response based on the input message.
        """
        for msg in self.messages:
            if msg in message:
                return random.choice(self.responses)
        return "I'm sorry, I didn't understand your message."
    def chatbot(state: MessagesState):
        # Add system message if not already present

システムメッセージがまだ存在しない場合は追加します。

if 'system' not in messages:
    messages.append({'role': 'system', 'content': system_message})
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成する

import networkx as nx

G = nx.Graph()

# 頂点を追加する
G.add_nodes_from([1, 2, 3])

# エッジを追加する
G.add_edges_from([(1, 2), (1, 3), (2, 3)])

# 頂点の位置を設定する
pos = {1: (0, 0), 2: (1, 1), 3: (-1, 1)}

# グラフを描画する
nx.draw(G, pos, with_labels=True)
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加する

日本語訳:

以下のテキストを英語から日本語に翻訳します。半角英数字の前後には半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残します。

`kubectl get nodes` を実行して、クラスターのノードを一覧表示します。出力例を次に示します。

```
NAME                                STATUS   ROLES    AGE   VERSION
gke-my-cluster-default-pool-7e5cd2fe-0d9b   Ready    <none>   1h    v1.19.9-gke.1900
gke-my-cluster-default-pool-7e5cd2fe-z6xh   Ready    <none>   1h    v1.19.9-gke.1900
```

新しいノードを追加するには、`gcloud` コマンドを使用します。`gcloud container clusters resize` コマンドを使って、ノードの数を増やすことができます。

```bash
gcloud container clusters resize my-cluster --node-pool default-pool --num-nodes 4
```

この例では、 `my-cluster` という名前のクラスターの `default-pool` ノードプールのノード数を 4 に増やしています。`--num-nodes` フラグを使って、ノード数を指定します。

ノードの追加が完了すると、`kubectl get nodes` で新しいノードが一覧に表示されます。

```
NAME                                STATUS   ROLES    AGE   VERSION
gke-my-cluster-default-pool-7e5cd2fe-0d9b   Ready    <none>   1h    v1.19.9-gke.1900
gke-my-cluster-default-pool-7e5cd2fe-z6xh   Ready    <none>   1h    v1.19.9-gke.1900
gke-my-cluster-default-pool-7e5cd2fe-9jwk   Ready    <none>   1m    v1.19.9-gke.1900
gke-my-cluster-default-pool-7e5cd2fe-k3x8   Ready    <none>   1m    v1.19.9-gke.1900
```
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges

graph.add_edge( 'A', 'B', weight=4 )
graph.add_edge( 'B', 'C', weight=8 )
graph.add_edge( 'A', 'C', weight=7 )

日本語訳:

# 辺を追加

graph.add_edge( 'A', 'B', weight=4 ) 
graph.add_edge( 'B', 'C', weight=8 )
graph.add_edge( 'A', 'C', weight=7 )
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point

エントリポイントを設定します。

ENTRYPOINT [ "python3" ]

日本語訳:

# エントリポイントを設定します

ENTRYPOINT [ "python3" ]
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイルする

日本語訳:

コンパイラ = tf.lite.TFLiteConverter.from_saved_model("saved_model")
コンパイラ.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = コンパイラ.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)
    return graph_builder.compile()

# エージェントを初期化する

agent = Agent()

# Set the agent's name
agent.name = "JARVIS"

# Train the agent
agent.train(data_path="path/to/training/data")

# Deploy the agent
agent.deploy(environment="production")

print(f"Agent {agent.name} has been successfully deployed.")
agent = create_agent()

def langgraph_bedrock(payload):
    以下が日本語訳になります。

"""
 agent を payload で呼び出す
"""
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph

LangGraph が期待する形式で入力を作成します。

半角英数字の前後には半角スペースを挿入します。 コード や コマンド 、 変数名 、 関数名 などの 技術的な用語 は 翻訳せず 、 そのまま残します。
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content

最終的なメッセージの内容を抽出します。

content = message.content

if message.reference is not None:
    referenced_message = await message.channel.fetch_message(message.reference.message_id)
    content = referenced_message.content

print(f'Message content: {content}')
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_bedrock(json.loads(args.payload))
    print(response)

#### ローカルエージェントの呼び出し

日本語訳:

ローカルエージェントを呼び出すには、 `npm run start:agent` コマンドを実行します。これにより、 agent.js ファイルが実行され、ローカルサーバーが起動します。

エージェントは、 `http://localhost:8000` でアクセスできます。エージェントの API エンドポイントは `/api/agent` です。

エージェントに質問を送信するには、次のようなリクエストを行います:

```
POST /api/agent HTTP/1.1
Content-Type: application/json

{
  "question": "質問の内容"
}
```

エージェントからの応答は JSON 形式で返されます:

```json
{
  "answer": "回答の内容"
}
```

エージェントの振る舞いは、 agent.js ファイルで定義されています。デフォルトでは、単純な質問応答システムとして実装されていますが、必要に応じてカスタマイズできます。

In [ ]:
!python langgraph_bedrock.py '{"prompt": "What is the weather now?"}'

## AgentCore Runtimeにエージェントを展開する準備

さて、私たちのエージェントをAgentCore Runtimeに展開しましょう。そのためには、以下の手順が必要です。

* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポートする
* `app = BedrockAgentCoreApp()` でコード内でアプリを初期化する
* 呼び出し関数に `@app.entrypoint` デコレータを付ける
* `app.run()` でAgentCoreRuntimeがエージェントの実行を制御する

### Amazon Bedrock モデルを使った LangGraph
Amazon Bedrock モデルを使った LangGraph から始めましょう。異なるフレームワークやモデルを使った他の例は、親ディレクトリにあります。

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# 計算ツールを作成

日本語訳:

# 計算ツールを作成

まず、 Python の組み込みモジュールである `tkinter` をインポートします。

```python
import tkinter as tk
```

次に、ウィンドウを作成し、タイトルを設定します。

```python
window = tk.Tk()
window.title("Calculator")
```

ディスプレイ用のエントリーウィジェットを作成し、グリッドレイアウトを使って配置します。

```python
display = tk.Entry(window, font=("Arial", 16), justify="right")
display.grid(row=0, column=0, columnspan=4, sticky="nsew")
```

関数 `add_digit()` は、ユーザーがボタンをクリックしたときに呼び出され、対応する数字をディスプレイに追加します。

```python
def add_digit(digit):
    value = display.get() + str(digit)
    display.delete(0, tk.END)
    display.insert(0, value)
```

`eval()` 関数を使って計算を実行する `calculate()` 関数を定義します。

```python
def calculate():
    result = str(eval(display.get()))
    display.delete(0, tk.END)
    display.insert(0, result)
```

ボタンウィジェットを作成し、それぞれのボタンに関数を割り当てます。

```python
buttons = []
for i in range(10):
    button = tk.Button(window, text=str(i), font=("Arial", 16), command=lambda x=i: add_digit(x))
    buttons.append(button)

operator_buttons = [
    tk.Button(window, text="+", font=("Arial", 16), command=lambda: add_digit("+")),
    tk.Button(window, text="-", font=("Arial", 16), command=lambda: add_digit("-")),
    tk.Button(window, text="*", font=("Arial", 16), command=lambda: add_digit("*")),
    tk.Button(window, text="/", font=("Arial", 16), command=lambda: add_digit("/")),
    tk.Button(window, text="=", font=("Arial", 16), command=calculate)
]
```

ボタンをグリッドレイアウトで配置します。

```python
row = 1
col = 0
for button in buttons:
    button.grid(row=row, column=col, sticky="nsew")
    col += 1
    if col > 2:
        col = 0
        row += 1

row = 5
col = 0
for button in operator_buttons:
    button.grid(row=row, column=col, sticky="nsew")
    col += 1
```

ウィンドウのメインループを開始します。

```python
window.mainloop()
```

これで、簡単な計算ツールが完成しました。数字ボタンをクリックすると対応する数字がディスプレイに表示され、演算子ボタンをクリックすると演算子が追加されます。「=」ボタンをクリックすると、 `eval()` 関数を使って計算が実行され、結果がディスプレイに表示されます。
@tool
def calculator(expression: str) -> str:
    ```
数学的な式の結果を計算します。

Args:
    expression: 文字列として与えられた数学的な式 (例: "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")

Returns:
    計算結果を文字列で返します
```
    try:
        # Define safe functions that can be used in expressions

# 式で使用できる安全な関数を定義

安全な関数とは、 eval() や exec() などの危険な組み込み関数を使用せずに、式の評価を行う関数のことです。以下に、いくつかの安全な関数の例を示します。

def safe_add(a, b):
    "Return the sum of a and b"
    return a + b

def safe_sub(a, b):
    "Return the difference between a and b"
    return a - b

def safe_mul(a, b):
    "Return the product of a and b"
    return a * b

def safe_div(a, b):
    "Return the quotient of a divided by b"
    if b == 0:
        raise ValueError("Division by zero")
    return a / b

これらの関数は、数値演算のみを行い、外部のデータや関数を読み込んだりしません。したがって、悪意のあるコードが実行されるリスクがありません。式の評価には、これらの安全な関数を使用することをお勧めします。
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # 数学関数

日本語訳:

以下は、 Python の数学関数の一覧です。

`math.ceil(x)` は、 `x` を超えない最小の整数を返します。例: `math.ceil(4.3)` は 5 を返します。

`math.floor(x)` は、 `x` 以下の最大の整数を返します。例: `math.floor(4.7)` は 4 を返します。

`math.fabs(x)` は、 `x` の絶対値を返します。例: `math.fabs(-4.5)` は 4.5 を返します。

`math.factorial(x)` は、 `x` の階乗を返します。例: `math.factorial(5)` は 120 を返します。

`math.gcd(a, b)` は、 `a` と `b` の最大公約数を返します。例: `math.gcd(8, 12)` は 4 を返します。

`math.pow(x, y)` は、 `x` の `y` 乗を返します。例: `math.pow(2, 3)` は 8 を返します。

`math.sqrt(x)` は、 `x` の平方根を返します。例: `math.sqrt(9)` は 3 を返します。
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)

# 基本的な演算子 (明示的な使用のため)

 x + y は、 x と y の和を計算します。 x - y は、 x から y を引いた値を計算します。 x * y は、 x と y の積を計算します。 x / y は、 x を y で割った商を計算します。 x // y は、 x を y で割った切り捨て整数の商を計算します。 x % y は、 x を y で割った余りを計算します。 -x は、 x の正負を反転させた値を計算します。 +x は、 x の値をそのまま返します (何もしません)。 abs(x) は、 x の絶対値を計算します。 int(x) は、 x を最も近い整数に変換します。 float(x) は、 x を浮動小数点数に変換します。 complex(re, im) は、複素数を作成します。 c.conjugate() は、 c の複素共役を計算します。 divmod(x, y) は、 (x // y, x % y) のペアを返します。 pow(x, y) は、 x の y 乗を計算します。 x ** y は、 pow(x, y) の短縮形です。
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # 式を安全に評価する

日本語訳:

この関数は、与えられた文字列を安全に評価します。まず、 `ast.parse()` を使って文字列を抽象構文木 (AST) に変換します。次に、 `ast.walk()` を使って AST を走査し、危険な構文 (組み込み関数の呼び出しなど) がないかチェックします。危険な構文が見つからない場合は、 `eval()` を使って式を評価し、その結果を返します。それ以外の場合は、 `ValueError` 例外をスローします。

この関数は、信頼できない入力を安全に評価する必要がある場合に使用できます。たとえば、ユーザーが入力した式を評価する必要がある Web アプリケーションなどで役立ちます。

使用例:

```python
result = safe_eval('2 + 3 * 4')  # 14
result = safe_eval('__import__("os").system("rm -rf /")')  # ValueError 例外をスロー
```
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# 独自の天気ツールを作成する

日本語訳:

独自の天気ツールを作成するには、以下の手順に従ってください。

1. まず、気象データを取得するための API キーを取得する必要があります。人気のある選択肢は OpenWeatherMap API と Dark Sky API です。

2. API キーを取得したら、プログラミング言語 (Python、JavaScript など) を選択し、API から天気データを取得するコードを記述します。API の使用方法については、それぞれのドキュメントを参照してください。

3. 次に、取得した天気データを処理し、必要な情報 (気温、湿度、風速など) を抽出します。

4. 最後に、抽出した情報を表示するユーザーインターフェース (Web アプリ、モバイルアプリ、コマンドラインツールなど) を作成します。

この手順を踏めば、独自の天気ツールを作成できます。ただし、API の利用規約を確認し、適切に利用することが重要です。また、より高度な機能 (天気予報の表示、位置情報の取得など) を追加することもできます。
@tool
def weather():
    以下が日本語訳になります。

"""Get weather"""

「 天気を取得する 」

技術的な用語は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。  # Dummy implementation

# ダミー実装

import random

# This is a dummy function to get a random number
def get_random_number():
    return random.randint(1, 100)

# This is a dummy class to store data
class DummyData:
    def __init__(self, value):
        self.value = value

    def get_value(self):
        return self.value

# Example usage
random_number = get_random_number()
print(f"Random number: {random_number}")

data = DummyData(42)
print(f"Data value: {data.get_value()}")
    return "sunny"

# エージェントを手動の LangGraph 構築で定義する

agent = gr.Agent.define_agent(
    "agent",
    memory=gr.Memory(
        memory_store=gr.RedisMemoryStore(redis_url="redis://redis:6379"),
        memory_session=gr.RedisSession(
            redis_url="redis://redis:6379", prefix="memory-session"
        ),
    ),
    skills=[
        gr.Skill(
            name="python_repl",
            description="Python REPL to execute code. Inputs: code. Outputs: result",
            fn=python_repl.python_repl,
            requirements=["openai"],
            load_env=True,
        ),
        gr.Skill(
            name="terminal",
            description="Execute terminal commands. Inputs: command. Outputs: result",
            fn=terminal.terminal,
            stream_output=True,
        ),
    ],
)
def create_agent():
    以下が日本語訳になります。

"""LangGraph エージェントの作成と構成"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)

大規模言語モデル (LLM) を初期化します (モデルとパラメーターは必要に応じて調整してください)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model

日本語訳:
# または好みの model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM

# ツールを LLM にバインドする

To allow the LLM to use tools, we need to bind them to it. This is done by creating an `AgentType` object and passing it a list of tools. Here is an example of binding the `python_repl` tool:

LLM がツールを使えるようにするには、ツールを LLM にバインドする必要があります。これは `AgentType` オブジェクトを作成し、ツールのリストを渡すことで行われます。ここに `python_repl` ツールをバインドする例を示します。

```python
from langchain.agents import AgentType
from langchain.python import PythonREPLTool

python_repl = PythonREPLTool()
agent = AgentType(tools=[python_repl])
```

Now the LLM can use the `python_repl` tool by referencing the tool name in its responses. For example, it could say `\python_repl\n<python code>` to execute some Python code.

これで、LLM はその応答でツール名を参照することにより、`python_repl` ツールを使用できるようになりました。例えば、`\python_repl\n<python code>` と言うことで、Python コードを実行できます。

You can bind as many tools as you want to an agent. Just pass a list of tool objects to the `AgentType` constructor.

エージェントにはいくつでもツールをバインドできます。`AgentType` コンストラクタにツールオブジェクトのリストを渡すだけです。
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message

システムメッセージ

The following code is used to initialize the database connection:

以下の コード は データベース接続を初期化するために使用されます:

```python
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="mydb",
    user="postgres",
    password="secret"
)
```

Please make sure to replace the `host`, `database`, `user`, and `password` values with your own configuration.

`host` 、 `database` 、 `user` 、および `password` の値は、ご自身の設定に置き換えてください。

If you encounter any issues, refer to the error log at `/var/log/myapp/error.log`.

問題が発生した場合は、 `/var/log/myapp/error.log` のエラーログを参照してください。
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node

chatbot ノードを定義します。

import random

responses = {
    "question": [
        "I don't have a great answer for that.",
        "I don't know how to respond to that.",
        "そうですね、私にもよくわかりません。"
    ],
    "statement": [
        "I can't really engage with statements like that.",
        "I'm not sure how to respond to that.",
        "はい、そうですね。"
    ]
}

class Chatbot:
    def respond(self, message):
        message = message.lower()
        if message.endswith("?"):
            response = random.choice(responses["question"])
        else:
            response = random.choice(responses["statement"])
        return response

chatbot = Chatbot()

print("Hi, I'm a chatbot. How can I help you today?")
while True:
    message = input("> ")
    if message.lower() == "quit":
        break
    print(chatbot.respond(message))
    def chatbot(state: MessagesState):
        # Add system message if not already present

システムメッセージがまだ存在しない場合は追加します。

if 'system' not in messages:
    messages.append({'role': 'system', 'content': system_message})
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成する

import networkx as nx

G = nx.Graph()

# 頂点を追加する
G.add_nodes_from([1, 2, 3])

# エッジを追加する
G.add_edges_from([(1, 2), (1, 3), (2, 3)])

# 頂点の位置を設定する
pos = {1: (0, 0), 2: (1, 1), 3: (-1, 1)}

# グラフを描画する
nx.draw(G, pos, with_labels=True)
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加する

日本語訳:

以下のテキストを英語から日本語に翻訳します。半角英数字の前後には半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残します。

`kubectl get nodes` を実行して、クラスターのノードを一覧表示します。出力例を次に示します。

```
NAME                                STATUS   ROLES    AGE   VERSION
gke-my-cluster-default-pool-7e5cd2fe-0d9b   Ready    <none>   1h    v1.19.9-gke.1900
gke-my-cluster-default-pool-7e5cd2fe-z6xh   Ready    <none>   1h    v1.19.9-gke.1900
```

新しいノードを追加するには、`gcloud` コマンドを使用します。`gcloud container clusters resize` コマンドを使って、ノードの数を増やすことができます。

```bash
gcloud container clusters resize my-cluster --node-pool default-pool --num-nodes 4
```

この例では、 `my-cluster` という名前のクラスターの `default-pool` ノードプールのノード数を 4 に増やしています。`--num-nodes` フラグを使って、ノード数を指定します。

ノードの追加が完了したら、再度 `kubectl get nodes` を実行して、新しいノードが追加されたことを確認できます。
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges

graph.add_edge( 'A', 'B', weight=4 )
graph.add_edge( 'B', 'C', weight=8 )
graph.add_edge( 'A', 'C', weight=7 )

日本語訳:

# エッジを追加

graph.add_edge( 'A', 'B', weight=4 ) 
graph.add_edge( 'B', 'C', weight=8 )
graph.add_edge( 'A', 'C', weight=7 )
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point

エントリーポイントを設定します。

ENTRYPOINT [ "python3" ]

日本語訳:

# エントリーポイントを設定します

ENTRYPOINT [ "python3" ]
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイルする

日本語訳:

コンパイラ = tf.lite.TFLiteConverter.from_saved_model("model.tflite")
コンパイラ.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = コンパイラ.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite モデルが model.tflite に保存されました。")
    return graph_builder.compile()

# エージェントを初期化する

agent = Agent()

# Set the agent's name
agent.name = "JARVIS"

# Train the agent
agent.train(data_path="path/to/training/data")

# Deploy the agent
agent.deploy(environment="production")

print(f"Agent {agent.name} has been successfully deployed.")
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    以下が日本語訳になります。

"""
 agent に payload を渡して実行する
"""
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph

LangGraph が期待する形式で入力を作成します。

半角英数字の前後には半角スペースを挿入します。 コード や コマンド 、 変数名 、 関数名 などの技術的な用語は翻訳せず、そのまま残します。
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content

最終的なメッセージの内容を抽出します。

message_content = message.content

if message.reference is not None:
    # This is a reply
    original_message = await message.channel.fetch_message(message.reference.message_id)
    message_content = f"Reply to {original_message.author.display_name}: {message_content}"

print(message_content)
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

## 裏側で何が起こっているのか?

`BedrockAgentCoreApp` を使用すると、自動的に以下のことが行われます。

* ポート 8080 でリッスンする HTTP サーバーを作成します
* エージェントの要件を処理するために必要な `/invocations` エンドポイントを実装します
* ヘルスチェック (非同期エージェントにとって非常に重要) のための `/ping` エンドポイントを実装します
* 適切なコンテンツタイプと応答フォーマットを処理します
* AWS 標準に従ってエラー処理を管理します

## AgentCore ランタイムにエージェントをデプロイする

`CreateAgentRuntime` 操作は包括的な構成オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するために、プロトコル設定 (HTTP、MCP) と認証メカニズムを構成することもできます。

**注:** オペレーションのベストプラクティスは、コードをコンテナにパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore ランタイムにデプロイします。

### ランタイムロールの作成

始める前に、AgentCore Runtimeのための IAM ロールを作成しましょう。あらかじめ開発された utils 関数を使用して行います。

日本語訳:
### ランタイムロールの作成

始める前に、AgentCore Runtime 用の IAM ロールを作成しましょう。あなたのために事前に開発された utils 関数を使って行います。

In [ ]:
import sys
import os
import json
import boto3

# Get the current notebook's directory

現在のノートブックのディレクトリを取得します。

import os
cwd = os.getcwd()
print( "Current working directory is: " + cwd )

# Print the current working directory
print( "Current working directory is: " + cwd )

# Change directory
os.chdir( '/path/to/new/directory' )

# Get the new current working directory
new_cwd = os.getcwd()
print( "New current working directory is: " + new_cwd )

# List files in the new directory
files = os.listdir( new_cwd )
print( "Files in %s:" % new_cwd )
for file in files:
    print( file )
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Navigate up to the utils.py location

utils.py の場所まで上へ移動します。
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]

日本語訳:

# sys.pathに追加する

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入する
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語はそのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="langgraph_bedrock"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、AgentCore Runtime デプロイをエントリポイント、作成した実行ロール、および要件ファイルで設定します。また、スターターキットを設定して、起動時に Amazon ECR リポジトリを自動的に作成するようにします。

設定ステップでは、アプリケーションコードに基づいて Dockerfile が生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

### AgentCore Runtimeへのエージェントの起動

Dockerfileができたので、次はAgentCore Runtimeにエージェントを起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### AgentCore Runtimeのステータスを確認する
AgentCore Runtimeをデプロイしたので、次はそのデプロイ状況を確認しましょう

日本語訳:

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtimeの呼び出し

最後に、ペイロードを使って AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response

### 呼び出し結果の処理

アプリケーションに組み込むために、呼び出し結果を処理することができます。

日本語訳:

### 呼び出し結果の処理

私たちは今や、アプリケーションに組み込むために、呼び出し結果を処理することができます。

In [ ]:
from IPython.display import Markdown, display
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### AgentCore Runtimeをboto3で呼び出す

AgentCore Runtimeが作成されたので、任意のAWS SDKでそれを呼び出すことができます。例えば、boto3の `invoke_agent_runtime` メソッドを使用できます。

日本語訳:
### AgentCore Runtimeを boto3 で呼び出す

AgentCore Runtime が作成されたので、任意の AWS SDK でそれを呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ (オプション)

作成した AgentCore Runtime をクリーンアップしましょう。

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_role_name,
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_role_name
)

# Congratulations!

日本語訳:

おめでとうございます!